# Lead Scoring Case Study

- Understand the valrous columns
- Missing value imputation
- check data imbalance
- Outlier treatment
- Dummy variable creation 
- Test train split
- Standardisation
- Build model
- Feature elimination
- Model evaluation

- Attach a lead score for each row
- Find a cut off for hot-lead

## Functions

In [83]:
import importlib

import pandas as pd

import matplotlib.pyplot as plt

import utils

## Read data set

In [84]:
df = pd.read_csv("Leads.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

### Rename, Group, Reorder columns

In [86]:
# Rename the columns that have long names or have spaces
df.rename(columns={'Prospect ID':'ProspectID'}, inplace=True)
df.rename(columns={'What is your current occupation':'Occupation', 'What matters most to you in choosing a course':'Reason'}, inplace=True)
df.rename(columns={'How did you hear about X Education':'Found', 'Search':'AdSearch', 'Magazine':'AdMagazine', 'Newspaper Article':'AdNewspaperArticle',\
                    'Newspaper':'AdNewspaper', 'X Education Forums':'AdForums','Digital Advertisement':'AdDigital', 'Through Recommendations':'Recommendations' }, inplace=True)
df.rename(columns={'Do Not Email':"NoEmail", 'Do Not Call':"NoCall",'Receive More Updates About Our Courses':"CourseUpdates", \
                    'Update me on Supply Chain Content':"SCCUpdates", 'Get updates on DM Content':"DMCUpdates",'A free copy of Mastering The Interview':"FreeCopy",\
                    'I agree to pay the amount through cheque':"PayAmount"}, inplace=True)
df.rename(columns={'Page Views Per Visit':'PageViews', 'Total Time Spent on Website':'TimeSpent','Last Activity':'LastActivity','Last Notable Activity':'LastNotableActivity'}, inplace=True)
df.rename(columns={'Lead Number':'LeadNumber', 'Lead Origin':'LeadOrigin', 'Lead Source': 'LeadSource','Lead Quality':'LeadQuality', 'Lead Profile':'LeadProfile'}, inplace=True)
df.rename(columns={'Asymmetrique Activity Index':'ActivityIndex', 'Asymmetrique Profile Index':'ProfileIndex', 'Asymmetrique Activity Score':'ActivityScore', \
                    'Asymmetrique Profile Score':'ProfileScore'}, inplace=True)
#df.rename(columns={}, inplace=True)

# Group the columns
COLS_ID         = ['ProspectID'] # customer id
COLS_LOC        = ['Country','City']

COLS_PROF       = ['Occupation', 'Reason','Specialization']
COLS_AD         = ['Found', 'AdSearch', 'AdMagazine', 'AdNewspaperArticle', 'AdForums', 'AdNewspaper', 'AdDigital', 'Recommendations']
COLS_PREF       = ['NoEmail', 'NoCall','CourseUpdates', 'SCCUpdates', 'DMCUpdates','FreeCopy','PayAmount']

COLS_ACTIVTIY   = ['TotalVisits', 'PageViews', 'TimeSpent','LastActivity', 'LastNotableActivity']

COLS_LEAD       = ['LeadNumber', 'LeadOrigin', 'LeadSource','LeadQuality',  'LeadProfile', 'Tags']
COLS_INDEXSCORE = ['ActivityIndex', 'ProfileIndex', 'ActivityScore', 'ProfileScore']

COLS_TARGET     = ['Converted']

df = df[COLS_ID + COLS_LOC + COLS_PROF + COLS_AD + COLS_PREF + COLS_ACTIVTIY + COLS_LEAD + COLS_INDEXSCORE + COLS_TARGET]

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ProspectID           9240 non-null   object 
 1   Country              6779 non-null   object 
 2   City                 7820 non-null   object 
 3   Occupation           6550 non-null   object 
 4   Reason               6531 non-null   object 
 5   Specialization       7802 non-null   object 
 6   Found                7033 non-null   object 
 7   AdSearch             9240 non-null   object 
 8   AdMagazine           9240 non-null   object 
 9   AdNewspaperArticle   9240 non-null   object 
 10  AdForums             9240 non-null   object 
 11  AdNewspaper          9240 non-null   object 
 12  AdDigital            9240 non-null   object 
 13  Recommendations      9240 non-null   object 
 14  NoEmail              9240 non-null   object 
 15  NoCall               9240 non-null   o

,ProspectID,Country,City,Occupation,Reason,Specialization,Found,AdSearch,AdMagazine,AdNewspaperArticle,...,LeadOrigin,LeadSource,LeadQuality,LeadProfile,Tags,ActivityIndex,ProfileIndex,ActivityScore,ProfileScore,Converted
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,NaN,Select,Unemployed,Better Career Prospects,Select,Select,No,No,No,...,API,Olark Chat,Low in Relevance,Select,Interested in other courses,02.Medium,02.Medium,15.0,15.0,0
1,2a272436-5132-4136-86fa-dcc88c88f482,India,Select,Unemployed,Better Career Prospects,Select,Select,No,No,No,...,API,Organic Search,NaN,Select,Ringing,02.Medium,02.Medium,15.0,15.0,0
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,India,Mumbai,Student,Better Career Prospects,Business Administration,Select,No,No,No,...,Landing Page Submission,Direct Traffic,Might be,Potential Lead,Will revert after reading the email,02.Medium,01.High,14.0,20.0,1
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,India,Mumbai,Unemployed,Better Career Prospects,Media and Advertising,Word Of Mouth,No,No,No,...,Landing Page Submission,Direct Traffic,Not Sure,Select,Ringing,02.Medium,01.High,13.0,17.0,0
4,3256f628-e534-4826-9d63-4a8b88782852,India,Mumbai,Unemployed,Better Career Prospects,Select,Other,No,No,No,...,Landing Page Submission,Google,Might be,Select,Will revert after reading the email,02.Medium,01.High,15.0,18.0,1


In [87]:
COLUMNS = df.columns.tolist(); colDetails(COLUMNS, "COLUMNS")

COLUMNS_CAT = df.select_dtypes(include=['object']).columns; 
COLUMNS_CAT = list(set(COLUMNS_CAT) - set(['Prospect ID']))
colDetails(COLUMNS_CAT, "COLUMNS_CAT")

COLUMNS_NULL = df[df.columns[df.isnull().any()]].columns.tolist(); colDetails(COLUMNS_NULL, "COLUMNS_NULL")


COLUMNS: 37 ['ProspectID', 'Country', 'City', 'Occupation', 'Reason', 'Specialization', 'Found', 'AdSearch', 'AdMagazine', 'AdNewspaperArticle', 'AdForums', 'AdNewspaper', 'AdDigital', 'Recommendations', 'NoEmail', 'NoCall', 'CourseUpdates', 'SCCUpdates', 'DMCUpdates', 'FreeCopy', 'PayAmount', 'TotalVisits', 'PageViews', 'TimeSpent', 'LastActivity', 'LastNotableActivity', 'LeadNumber', 'LeadOrigin', 'LeadSource', 'LeadQuality', 'LeadProfile', 'Tags', 'ActivityIndex', 'ProfileIndex', 'ActivityScore', 'ProfileScore', 'Converted']

COLUMNS_CAT: 30 ['NoCall', 'AdMagazine', 'AdForums', 'AdDigital', 'LastNotableActivity', 'LeadSource', 'LastActivity', 'Recommendations', 'FreeCopy', 'DMCUpdates', 'LeadQuality', 'LeadOrigin', 'Tags', 'LeadProfile', 'ProfileIndex', 'AdNewspaper', 'AdSearch', 'AdNewspaperArticle', 'Specialization', 'Found', 'PayAmount', 'ActivityIndex', 'City', 'ProspectID', 'Occupation', 'Country', 'Reason', 'SCCUpdates', 'NoEmail', 'CourseUpdates']

COLUMNS_NULL: 17 ['Country

### IDs
Identifiers of customer and lead
- Drop them for the model building

In [88]:
df[COLS_ID][:5]

,ProspectID
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620
1,2a272436-5132-4136-86fa-dcc88c88f482
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc
4,3256f628-e534-4826-9d63-4a8b88782852


In [91]:
importlib.reload(utils) 
for col in COLS_ID:
    utils.colInfo1(df, col, showValues=False)

ProspectID: object  :    0, 0.0% : 


- The prospect Ids are all unique and no missing values.
- This need not be part of the model

### LOC

In [92]:
df[COLS_LOC][:5]

,Country,City
0,NaN,Select
1,India,Select
2,India,Mumbai
3,India,Mumbai
4,India,Mumbai


In [94]:
for col in COLS_LOC:
    utils.colInfo1(df, col)

Country   : object  : 2461, 26.6% :   39, 0.4% : [nan, 'India', 'Russia', 'Kuwait', 'Oman', 'United Arab Emirates', 'United States', 'Australia', 'United Kingdom', 'Bahrain', 'Ghana', 'Singapore', 'Qatar', 'Saudi Arabia', 'Belgium', 'France', 'Sri Lanka', 'China', 'Canada', 'Netherlands', 'Sweden', 'Nigeria', 'Hong Kong', 'Germany', 'Asia/Pacific Region', 'Uganda', 'Kenya', 'Italy', 'South Africa', 'Tanzania', 'unknown', 'Malaysia', 'Liberia', 'Switzerland', 'Denmark', 'Philippines', 'Bangladesh', 'Vietnam', 'Indonesia']
City      : object  : 1420, 15.4% :    8, 0.1% : ['Select', 'Mumbai', nan, 'Thane & Outskirts', 'Other Metro Cities', 'Other Cities', 'Other Cities of Maharashtra', 'Tier II Cities']


| Column Name | Missing value handling| a |
|---|---|---|
| Country | Has missing values. Based on city some countries could be filled | |
| City | Has msising values & Select. Select should be replaced by null. | |

### PROF

In [95]:
df[COLS_PROF][:5]

,Occupation,Reason,Specialization
0,Unemployed,Better Career Prospects,Select
1,Unemployed,Better Career Prospects,Select
2,Student,Better Career Prospects,Business Administration
3,Unemployed,Better Career Prospects,Media and Advertising
4,Unemployed,Better Career Prospects,Select


In [97]:
importlib.reload(utils) 
utils.showColumns(df, COLS_PROF)

Occupation: object  : 2690, 29.1% :    7, 0.1% : ['Unemployed', 'Student', nan, 'Working Professional', 'Businessman', 'Other', 'Housewife']
Reason    : object  : 2709, 29.3% :    4, 0.0% : ['Better Career Prospects', nan, 'Flexibility & Convenience', 'Other']
Specialization: object  : 1438, 15.6% :   20, 0.2% : ['Select', 'Business Administration', 'Media and Advertising', nan, 'Supply Chain Management', 'IT Projects Management', 'Finance Management', 'Travel and Tourism', 'Human Resource Management', 'Marketing Management', 'Banking, Investment And Insurance', 'International Business', 'E-COMMERCE', 'Operations Management', 'Retail Management', 'Services Excellence', 'Hospitality Management', 'Rural and Agribusiness', 'Healthcare Management', 'E-Business']


| Column Name | Missing values? | Select? | Handling |
|---|---|---|---|
| Occupation | 29% | No | Replace NaN with 'Unknown' |
| Reason | 29% | No | Replace NaN with 'Unknown' |
| Specialization | 15.6% | Yes | Replace Select with NaN. Replace NaN with 'Others' |

### AD

In [98]:
utils.showColumns(df, COLS_AD)

Found     : object  : 2207, 23.9% :   11, 0.1% : ['Select', 'Word Of Mouth', 'Other', nan, 'Online Search', 'Multiple Sources', 'Advertisements', 'Student of SomeSchool', 'Email', 'Social Media', 'SMS']
AdSearch  : object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
AdMagazine: object  :    0, 0.0% :    1, 0.0% : ['No']
AdNewspaperArticle: object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
AdForums  : object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
AdNewspaper: object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
AdDigital : object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
Recommendations: object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']


| Column Name | Missing values? | Select? | Handling |
|---|---|---|---|
| Found | 23.9% | Yes | Replace NaN with 'Unknown' or Other?|
| AdXXX | - | - | - |
| Recommendations | - | - | - |

### COLUMNS_PREF
Preferences of the lead

In [99]:
utils.showColumns(df, COLS_PREF)

NoEmail   : object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
NoCall    : object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
CourseUpdates: object  :    0, 0.0% :    1, 0.0% : ['No']
SCCUpdates: object  :    0, 0.0% :    1, 0.0% : ['No']
DMCUpdates: object  :    0, 0.0% :    1, 0.0% : ['No']
FreeCopy  : object  :    0, 0.0% :    2, 0.0% : ['No', 'Yes']
PayAmount : object  :    0, 0.0% :    1, 0.0% : ['No']


| Column Name | Missing values? | Select? | Handling |
|---|---|---|---|
| XXX |  - | - | - |

In [ ]:
print("Do Not Email = No\n", df[df['Do Not Email']=='No']['Converted'].value_counts())
print("Do Not Email = Yes\n", df[df['Do Not Email']=='Yes']['Converted'].value_counts())

In [ ]:
print("Do Not Call = No\n", df[df['Do Not Call']=='No']['Converted'].value_counts())
print("Do Not Call = Yes\n", df[df['Do Not Call']=='Yes']['Converted'].value_counts())

In [ ]:
df[df['Do Not Email']=='Yes']['Converted']

### ACTIVITY
Activity of the lead - Indicator of candidate's interest.
- Visits, Time spent: Higher the value, higher is the interest
- Last Activity & Last Notable Activity share the same values except 'Converted to Lead' of Last Activity
- Visits to the site and time spent on the website

In [105]:
importlib.reload(utils)
utils.showColumns(df, COLS_ACTIVTIY)

TotalVisits: float64 :  137, 1.5% : 3 [0.0, 1.0, 3.0, 5.0, 251.0] 
PageViews : float64 :  137, 1.5% : 2 [0.0, 1.0, 2.0, 3.0, 55.0] 
TimeSpent : int64   :    0, 0.0% : 488 [0.0, 12.0, 248.0, 936.0, 2272.0] 
LastActivity: object  :  103, 1.1% :   18, 0.2% : ['nan', 'Approached upfront', 'Converted to Lead', 'Email Bounced', 'Email Link Clicked', 'Email Marked Spam', 'Email Opened', 'Email Received', 'Form Submitted on Website', 'Had a Phone Conversation', 'Olark Chat Conversation', 'Page Visited on Website', 'Resubscribed to emails', 'SMS Sent', 'Unreachable', 'Unsubscribed', 'View in browser link Clicked', 'Visited Booth in Tradeshow']
LastNotableActivity: object  :    0, 0.0% :   16, 0.2% : ['Approached upfront', 'Email Bounced', 'Email Link Clicked', 'Email Marked Spam', 'Email Opened', 'Email Received', 'Form Submitted on Website', 'Had a Phone Conversation', 'Modified', 'Olark Chat Conversation', 'Page Visited on Website', 'Resubscribed to emails', 'SMS Sent', 'Unreachable', 'Unsubs

| Column Name | Missing values? | Select? | Handling |
|---|---|---|---|
| TotalVisits | 1.5% | - | Remove the rows with null values |
| PageViews | 1.5% | - | Remove the rows with null values |
| TimeSpent | -  | - | - |
| LastActivity |  1.1% | -  | Remove the rows with null values |
| LastNotableActivity | - | - | - |

Note: LastActivity & LastNotableActivity differ in 2 values - nan,  'Converted to Lead'

In [ ]:
plt.figure(figsize=(20, 6), dpi=80)
ax = plt.subplot(1, 2, 1); 
df[df['Converted']==1].groupby('Last Activity').agg({'Last Activity':'count'}). \
        rename(columns={'Last Activity':'count'}).sort_values(by='count', ascending=False).plot.bar(ax=ax)#, ax=ax)
ax.set_title(f"Converted==1")

ax = plt.subplot(1, 2, 2); 
df[df['Converted']==0].groupby('Last Activity').agg({'Last Activity':'count'}). \
        rename(columns={'Last Activity':'count'}).sort_values(by='count', ascending=False).plot.bar(ax=ax)#, ax=ax)
ax.set_title(f"Converted==0")

In [ ]:
plt.figure(figsize=(20, 4), dpi=80)
ax = plt.subplot(2, 3, 1); df.boxplot(column='TotalVisits', by='Converted', ax=ax)
ax = plt.subplot(2, 3, 2); df.boxplot(column='Page Views Per Visit', by='Converted', ax=ax)
ax = plt.subplot(2, 3, 3); df.boxplot(column='Total Time Spent on Website', by='Converted', ax=ax)

#ax = plt.subplot(2, 3, 4); df.plot.bar(x='Last Activity', by='Converted', ax=ax)
#ax = plt.subplot(2, 3, 5); df.plot.bar(x='Last Notable Activity', by='Converted', ax=ax)
#ax = plt.subplot(2, 3, 6); df.plot.bar(column='Total Time Spent on Website', by='Converted', ax=ax)

### INDEXSCORE

In [106]:
utils.showColumns(df, COLS_INDEXSCORE)

ActivityIndex: object  : 4218, 45.6% :    4, 0.0% : ['nan', '01.High', '02.Medium', '03.Low']
ProfileIndex: object  : 4218, 45.6% :    4, 0.0% : ['nan', '01.High', '02.Medium', '03.Low']
ActivityScore: float64 : 4218, 45.6% : 14 [7.0, 14.0, 14.0, 15.0, 18.0] 
ProfileScore: float64 : 4218, 45.6% : 16 [11.0, 15.0, 16.0, 18.0, 20.0] 


| Column Name | Missing values? | Select? | Handling |
|---|---|---|---|
| ActivityIndex | 45.6%  | - | TODO |
| ProfileIndex | 45.6%  | - | TODO |
| ActivityScore | 45.6%  | - | TODO |
| ProfileScore | 45.6%  | - | TODO |

### LEAD
Lead related data - Where was the customer identified as a lead? Source, quality and level/profile of the lead


In [107]:
utils.showColumns(df, COLS_LEAD)

LeadNumber: int64   :    0, 0.0% : 617188 [579533.0, 596484.5, 615479.0, 637387.25, 660737.0] 
LeadOrigin: object  :    0, 0.0% :    5, 0.1% : ['API', 'Landing Page Submission', 'Lead Add Form', 'Lead Import', 'Quick Add Form']
LeadSource: object  :   36, 0.4% :   22, 0.2% : ['nan', 'Click2call', 'Direct Traffic', 'Facebook', 'Google', 'Live Chat', 'NC_EDM', 'Olark Chat', 'Organic Search', 'Pay per Click Ads', 'Press_Release', 'Reference', 'Referral Sites', 'Social Media', 'WeLearn', 'Welingak Website', 'bing', 'blog', 'google', 'testone', 'welearnblog_Home', 'youtubechannel']
LeadQuality: object  : 4767, 51.6% :    6, 0.1% : ['nan', 'High in Relevance', 'Low in Relevance', 'Might be', 'Not Sure', 'Worst']
LeadProfile: object  : 2709, 29.3% :    7, 0.1% : ['nan', 'Dual Specialization Student', 'Lateral Student', 'Other Leads', 'Potential Lead', 'Select', 'Student of SomeSchool']
Tags      : object  : 3353, 36.3% :   27, 0.3% : ['nan', 'Already a student', 'Busy', 'Closed by Horizzon', 

| Column Name | Missing values? | Select? | Handling |
|---|---|---|---|
| LeadNumber | - | - | - |
| LeadOrigin | - | - | - |
| LeadSource | 0.4% | - | Remove the rows with null values |
| LeadQuality | 51.6% | - | |
| LeadProfile | 29.3% | Yes | |
| Tags | 36.3% | | |


In [ ]:
df[df['Converted']==1][COLS_LEAD + ['Lead Number']]

### How are customers who are converted to paid customers?
What are the characteristics of the customers who are converted and not converted?

In [ ]:
TOTAL = df.shape[0]
print(TOTAL)

for value in df['Converted'].value_counts().index:
    count = df['Converted'].value_counts()[value]
    print(f"{value:2}: {count} : {100 * count/TOTAL:6.2f}% ", end='')
    print()


In [ ]:
df[df['Converted']==1][COLS_ASYM]

In [ ]:
df[COLS_ASYM].plot.box(by='Converted', figsize = (10, 6));plt.show()

In [ ]:
plt.figure(figsize=(20, 6), dpi=80)

ax = plt.subplot(1, 2, 1); df.boxplot(column='Asymmetrique Activity Score', by='Converted', ax=ax)
ax=plt.subplot(1, 2, 2); df.boxplot(column='Asymmetrique Profile Score', by='Converted', ax=ax)

## MISSING VALUES

In [ ]:
df[df.columns[df.isnull().any()]].info()

In [ ]:
import missingno as msno
msno.matrix(df)

In [ ]:
msno.bar(df)

In [ ]:
TOTAL = df.shape[0]
print(TOTAL)
for col in COLUMNS_NULL:
    print(f"{col:50}: {df[col].dtype.name:8}: {df[col].isnull().sum():6d} : {100 * df[col].isnull().sum()/TOTAL:6.2f}% ", end='')
    #if df[col].dtype == 'object':
    #    print(f"{df[col].value_counts().index.tolist()}",end='')
    print()


In [ ]:
df['Search'].value_counts()

In [ ]:
df['Newspaper'].value_counts()

In [ ]:
COLUMNS_LEAD = ['Lead Origin', 'Lead Source', 'Lead Quality']
df[COLUMNS_LEAD]

In [ ]:
COLUMNS_ADS = ['Search', 'Magazine', 'Newspaper Article', 'X Education Forums', 'Newspaper', 'Digital Advertisement']
df[df[COLUMNS_ADS].isin(['Yes']).any(axis=1)][COLUMNS_ADS]

In [ ]:
df['Asymmetrique Activity Index'].value_counts()

In [ ]:
df['Asymmetrique Activity Score'].value_counts()

In [ ]:
df[df['Asymmetrique Activity Index'].isnull()][['Lead Source', 'Lead Quality', 'Lead Profile', 'Asymmetrique Activity Index', 'Asymmetrique Activity Score','Asymmetrique Profile Index','Asymmetrique Profile Score']]

In [ ]:
df['Lead Quality'].value_counts()

In [ ]:
df[df['Specialization']=='Select'].shape[0]

In [ ]:
for col in COLUMNS:
    nan = df[col].isnull().sum()
    select = df[df[col]=='Select'].shape[0]
    if nan > 0 or select > 0:
        print(f"{col:50}: {nan:6d} : {100 * nan/TOTAL:6.2f}% : {select:4d} : {100 * select/TOTAL:6.2f}% : {df[col].dtype}", end='')
        #if df[col].dtype == 'object':
        #    print(f"{df[col].value_counts().index.tolist()}",end='')
        print()

In [ ]:
print(df.shape[0])
for col in ['Lead Source', 'TotalVisits', 'Page Views Per Visit', 'Last Activity']:
    df.drop(df[df[col].isnull()].index, inplace=True)
print(df.shape[0])

In [ ]:
for col in COLUMNS:
    nan = df[col].isnull().sum()
    select = df[df[col]=='Select'].shape[0]
    if nan > 0 or select > 0:
        print(f"{col:50}: {nan:6d} : {100 * nan/TOTAL:6.2f}% : {select:4d} : {100 * select/TOTAL:6.2f}% : {df[col].dtype}", end='')
        #if df[col].dtype == 'object':
        #    print(f"{df[col].value_counts().index.tolist()}",end='')
        print()

In [ ]:
df['Country'].value_counts()
# Impute null values to unknown?

In [ ]:
df['City'].value_counts()

In [ ]:
df['Lead Profile'].value_counts()
# Impute it to Other Leads?

In [ ]:
df['Lead Quality'].value_counts()
# Impute it to 'Not Sure'??

In [ ]:
df['Tags'].value_counts()

In [ ]:
df['What matters most to you in choosing a course'].value_counts()
# TODO: Impute this with 'Other'?

In [ ]:
df['What is your current occupation'].value_counts()
# TODO: Impute this with 'Other'?

In [ ]:
df['How did you hear about X Education'].value_counts()
# TODO: Impute this with 'Other'?

In [ ]:
df['Prospect ID'].unique().shape[0]
# This can be dropped fro the model

In [ ]:
df['Lead Number'].unique().shape[0]
# This can be dropped fro the model

In [ ]:
df['Specialization'].value_counts()
# TODO: Impute this with ??

In [ ]:
df[(df['Country'].isnull()) & (~df['City'].isnull()) & (df['City']!='Select')][['City']].value_counts()

In [ ]:
import seaborn as sns

sns.heatmap(df.corr())

In [ ]:
sns.pairplot(data=df)

In [ ]:
df.plot.bar(x='Country')